<a href="https://colab.research.google.com/github/EnriqueFV/cad_dl/blob/main/Binary%20Ineptionv3%20sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch>=1.9
!pip install fastai>=2.7
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install torchvision
!pip install tqdm
!pip install wandb
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.5 MB/s eta 0:00:00


In [2]:
import wandb
import pandas as pd
import torch
import torchvision.models as tvmodels
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback
import cv2
import timm
from sklearn.metrics import ConfusionMatrixDisplay
from IPython.display import display, Markdown

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 28 16:10:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
WANDB_PROJECT = "cad_dl"
ENTITY = "iconet" # set this to team name if working in a team

RAW_DATA_AT = 'skin_data'
TEST_DATA_AT = 'test_data'
PROCESSED_DATA_AT = 'skin_data_proc'
TV_DATA_AT = 'train_val_data'
TABLE_DATA_AT = 'train_val_table'
TABLE_TEST_AT = 'test_table'

In [13]:
def get_df(processed_dataset_dir, is_test = False):
  df = pd.read_csv(processed_dataset_dir / 'data.csv')

  if not is_test:
    df = df[df.Stage != 'test'].reset_index(drop=True)
    df['is_valid'] = df.Stage == 'valid'
  else:
    df = df[df.Stage == 'test'].reset_index(drop=True)

  df["Label"] = [f'{f}' for f in df.Label.values]

  return df

def find_image_path(file_name, is_test = False):
    if not is_test:
      folders = ['nevus_train_images', 'others_train_images', 'nevus_val_images', 'others_val_images']
    else:
      folders = ['xxx_test_images']


    for folder in folders:
        potential_path = path / folder / f'{file_name}.jpg'
        if potential_path.exists():
            return potential_path

    return None  # If the path couldn't be determined

def get_data(df, bs=4, img_size=(128, 128), augment=True):
    block = DataBlock(blocks=(ImageBlock, CategoryBlock),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("Label"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=[*aug_transforms(),
                               Normalize.from_stats(*imagenet_stats)]
                 )
    return block.dataloaders(df, bs=bs)

def log_final_metrics(learn):
  scores = learn.validate()
  metric_names = ['final_loss'] + [f'final_{x.name}' for x in learn.metrics]
  final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
  for k,v in final_results.items():
      wandb.summary[k] = v

def train(config=None):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  print(f"Training on device: {device}")
  # Initialize a new wandb run
  with wandb.init(config=config):
      # If called by wandb.agent, as below,
      # this config will be set by Sweep Controller
      config = wandb.config

      set_seed(config.seed, reproducible=True)
      run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="training", config=config)

      dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

      metrics = [Precision(), Recall(), accuracy, F1Score(), MatthewsCorrCoef()]

      learn = vision_learner(dls, arch=getattr(tvmodels, config.arch), pretrained=config.pretrained, metrics=metrics)

      cbs = [WandbCallback(log_preds=True, log_model=True),
              SaveModelCallback(fname=f'run-{wandb.run.id}-model', monitor='accuracy')]

      learn.fit_one_cycle(config.epochs, config.lr, cbs=cbs)

      log_final_metrics(learn)

      wandb.finish()

def get_predictions(learner, test_dl=None, max_n=None):
    """Return the samples = (x,y) and outputs (model predictions decoded), and predictions (raw preds)"""
    test_dl = learner.dls.valid if test_dl is None else test_dl
    inputs, predictions, targets, outputs = learner.get_preds(
        dl=test_dl, with_input=True, with_decoded=True
    )
    x, y, samples, outputs = learner.dls.valid.show_results(
        tuplify(inputs) + tuplify(targets), outputs, show=False, max_n=max_n
    )
    return samples, outputs, predictions

def train_eval(config):

  run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="evaluation", tags=['staging'])

  artifact = run.use_artifact('iconet/model-registry/Skin Binary Classification:v0', type='model')

  artifact_dir = Path(artifact.download())

  _model_pth = artifact_dir.ls()[0]
  model_path = _model_pth.parent.absolute()/_model_pth.stem

  producer_run = artifact.logged_by()
  wandb.config.update(producer_run.config)
  config = wandb.config

  processed_dataset_dir = download_data()
  test_dir = download_images()
  test_df = get_df(processed_dataset_dir, is_test=True)
  test_df["image_fname"] = test_df["File_Name"].apply(find_image_path())
  test_dls = get_data(test_df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

  metrics = [Precision(), Recall(), accuracy(), F1Score(), MatthewsCorrCoef()]

  cbs = [WandbCallback(log_preds=True, log_model=True),
                SaveModelCallback(fname=f'run-{wandb.run.id}-model', monitor='accuracy')]

  learn = vision_learner(test_dls, arch=getattr(tvmodels, config.arch), pretrained=config.pretrained, metrics=metrics)

  learn.load(model_path)

  val_metrics = learn.validate(ds_idx=1, )
  test_metrics = learn.validate(ds_idx=0)

  val_metric_names = ['val_loss'] + [f'val_{x.name}' for x in learn.metrics]
  val_results = {val_metric_names[i] : val_metrics[i] for i in range(len(val_metric_names))}
  for k,v in val_results.items():
      wandb.summary[k] = v

  test_metric_names = ['test_loss'] + [f'test_{x.name}' for x in learn.metrics]
  test_results = {test_metric_names[i] : test_metrics[i] for i in range(len(test_metric_names))}
  for k,v in test_results.items():
      wandb.summary[k] = v

  val_probs, val_targs = learn.get_preds(ds_idx=1)
  val_preds = val_probs.argmax(dim=1)


  wandb.finish()



In [6]:
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY)
processed_data_at = wandb.use_artifact(f'{PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())

tv_data_at = wandb.use_artifact(f'{TV_DATA_AT}:latest')
path = Path(tv_data_at.download())
wandb.finish()


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb:   5 of 5 files downloaded.  
wandb: Downloading large artifact train_val_data:latest, 7008.60MB. 18991 files... 
wandb:   18991 of 18991 files downloaded.  
Done. 0:3:1.6


In [ ]:
df = get_df(processed_dataset_dir, is_test = False)

# Apply the function to create a new column with the correct paths
df["image_fname"] = df["File_Name"].apply(find_image_path)

sweep_config = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize',
        'name': 'accuracy'
        },
    'parameters': {
        'batch_size': {'values': [8, 32]},
        'epochs': {'values': [10, 15]},
        'lr': {
            'distribution': 'log_uniform_values',
            'max': 1e-2,
            'min': 1e-5
        },
        'augment': {'value': 'True'},
        'pretrained': {'value': 'True'},
        'img_size': {'value': (256, 256)},
        'seed': {'values': [42]},
        'arch': {
            'values': ['inception_v3']
            }
        },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 3
    }
}

sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)

wandb.agent(sweep_id, train)

Create sweep with ID: tjmmx84m
Sweep URL: https://wandb.ai/iconet/cad_dl/sweeps/tjmmx84m


wandb: Agent Starting Run: 2pjeozi6 with config:
wandb: 	arch: vgg11_bn
wandb: 	augment: True
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	img_size: [256, 256]
wandb: 	lr: 0.0003211105049969196
wandb: 	pretrained: True
wandb: 	seed: 42


Training on device: cuda


wandb: Currently logged in as: enriquefvrc (iconet). Use `wandb login --relogin` to force relogin


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

Training on device: cuda


wandb: Currently logged in as: enriquefvrc (iconet). Use `wandb login --relogin` to force relogin


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:06<00:00, 82.8MB/s]


epoch,train_loss,valid_loss,precision_score,recall_score,accuracy,f1_score,matthews_corrcoef,time
